MIT License

Copyright (c) 2021 Taiki Miyagawa and Akinori F. Ebihara

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Clip Videos
DATADIR/HMDB51clip79/ and DATADIR/HMDB51clip200/ will be created.

In [2]:
from glob import glob
import os, shutil
from copy import copy
import statistics
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
DATADIR = "Define this first. E.g., /data/t-miyagawa"

In [3]:
# Get videodir and numf
datadir = "{}/HMDB51png".format(DATADIR)
classdir = sorted(glob(datadir + "/*"))
classdir = [i + "/" for i in classdir]
classnames = [i[i.rfind("HMDB51png/") + 10 : -1] for i in classdir]
videodir = {
    k : 
    sorted(glob([v for v in classdir if v.find("/" + k + "/") != -1][0] + "/*"))
    for k in classnames}

numf = dict()
for k in classnames:
    v1 = videodir[k]
    v2 = [i.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]") for i in v1]
    numf[k] = [len(glob(_video + "/*.png")) for _video in v2]

# Smear the keys
numf_concat = []
for k in classnames:
    v = numf[k]
    numf_concat.extend(v)

videodir_concat = []
for k in classnames:
    v = videodir[k]
    videodir_concat.extend(v)
    
# Classwise num of frames
numf_classwise = []
for k in classnames:
    v = numf[k]
    v = sum(v)
    numf_classwise.append(v)
    
# Classwise num of videos (clips)
numv_classwise = []
for k in classnames:
    v = videodir[k]
    v = len(v)
    numv_classwise.append(v)
    
# Classwise num of unique videos (groups)
numuv_classwise = []
for k in classnames:
    v1 = videodir[k]
        # ['DATADIR/HMDB51png/wave/20060723sfjffbartsinger_wave_f_cm_np1_ba_med_0',
        #  'DATADIR/HMDB51png/wave/21_wave_u_nm_np1_fr_goo_5',
        #  'DATADIR/HMDB51png/wave/50_FIRST_DATES_wave_f_cm_np1_fr_med_0',
        #  'DATADIR/HMDB51png/wave/50_FIRST_DATES_wave_u_cm_np1_fr_goo_30',
        #  'DATADIR/HMDB51png/wave/50_FIRST_DATES_wave_u_cm_np1_fr_med_1',
        #  'DATADIR/HMDB51png/wave/50_FIRST_DATES_wave_u_cm_np1_fr_med_36',
    v2 = [i[i.rfind("/")+1:] for i in v1]
        # ['20060723sfjffbartsinger_wave_f_cm_np1_ba_med_0',
        #  '21_wave_u_nm_np1_fr_goo_5',
        #  '50_FIRST_DATES_wave_f_cm_np1_fr_med_0',
        #  '50_FIRST_DATES_wave_u_cm_np1_fr_goo_30',
        #  '50_FIRST_DATES_wave_u_cm_np1_fr_med_1',
        #  '50_FIRST_DATES_wave_u_cm_np1_fr_med_36',
    v3 = [i[:i.rfind(k)-1] for i in v2]
        # ['20060723sfjffbartsinger',
        #  '21',
        #  '50_FIRST_DATES',
        #  '50_FIRST_DATES',
        #  '50_FIRST_DATES',
        #  '50_FIRST_DATES',
    v4 = []
    for i in v3:
        if not i in v4:
            v4.append(i)
        # ['20060723sfjffbartsinger',
        #  '21',
        #  '50_FIRST_DATES',
    numuv_classwise.append(len(v4))
    

# """
# Returns:
#     classnames: List. Len = Num of classes. Names of classes in alphabetical order.
#
#     videodir: Dict. Paths to video directories. Each values (paths) are in alphabetical order of video names.
#     numf: Dict. Num of frames for each videos. Each values (integers) are in alphabetical order of video names.
#
#     numf_concat: List. Len = Num of total videos. Order is the same as `videoddir_concat`.
#     videodir_concat: List. Len = Num of total videos. Order is the same as `numf_concat`.
#
#     numf_classwise: List. Len = Num of classes. The classwise numbers of frames in alphabetical order of class names.
#     numv_classwise: List. Len = Num of classes. The classwise numbers of videos in alphabetical order of class names.
#     numuv_classwise: List. Len = Num of classes. The classwise numbers of unique videos (groups) in alphabetical order of class names.
# """

In [4]:
def repeat_func(paths_images, duration):
    """ Num of frames < duration.
    Args:
        paths_images: Paths to images in ascending order of frame numbers.
            E.g.,
            ['DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0001.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0002.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0003.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0004.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0005.png',...]       
    """
    num_repeat = duration // len(paths_images)
    remainder = duration % len(paths_images)
    
    # Create new video directory paths
    vpath = paths_images[0]
    vpath = vpath[: vpath.rfind("/")] # Original video directory path.
        # e.g., 'DATADIR/HMDB51png/YoYo/v_YoYo_g25_c05
    new_vpath = vpath.replace("HMDB51png", "HMDB51clip{}".format(duration)) + "_cc01"
        # e.g., 'DATADIR/HMDB51clip79/YoYo/v_YoYo_g25_c05_cc00
        
    # Create directories
    if not os.path.exists(new_vpath):
        os.makedirs(new_vpath)

    # Copy images
    rep_paths_images = copy(paths_images)
    for _ in range(num_repeat - 1):
        rep_paths_images.extend(paths_images)
    rep_paths_images.extend(paths_images[:remainder])

    new_paths_images = [new_vpath + "/{:04d}.png".format(i + 1) for i in range(duration)]

    assert len(rep_paths_images) == len(new_paths_images), "{} != {}".format(len(rep_paths_images), len(new_paths_images))
    for i, j in zip(rep_paths_images, new_paths_images):
        shutil.copy(i, j)

    print("Done: {}.".format(vpath))
            
            
            
def as_is_func(paths_images, duration):
    """ Num of frames = duration.
    Args:
        paths_images: Paths to images in ascending order of frame numbers.
            E.g.,
            ['DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0001.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0002.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0003.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0004.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0005.png',...]       
    """
    # Create new video directory paths
    vpath = paths_images[0]
    vpath = vpath[: vpath.rfind("/")] # Original video directory path.
        # e.g., 'DATADIR/HMDB51png/YoYo/v_YoYo_g25_c05
    new_vpath = vpath.replace("HMDB51png", "HMDB51clip{}".format(duration)) + "_cc01"
        # e.g., 'DATADIR/HMDB51clip79/YoYo/v_YoYo_g25_c05_cc00
        
    # Create directories
    if not os.path.exists(new_vpath):
        os.makedirs(new_vpath)

    # Copy images
    new_paths_images = [v.replace(vpath, new_vpath) for v in paths_images]

    for i, j in zip(paths_images, new_paths_images):
        shutil.copy(i, j)

    print("Done: {}.".format(vpath))

    
def clip_func(paths_images, duration):
    """ Num of frames > duration.
    Args:
        paths_images: Paths to images in ascending order of frame numbers.
            E.g.,
            ['DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0001.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0002.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0003.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0004.png',
             'DATADIR/HMDB51png/situp/5_Min_Tone_Abs_Workout_2__Fitness_Training_w__Tammy_situp_f_nm_np1_fr_goo_2/0005.png',...]
    """
    num_repeat =  len(paths_images) // duration
    remainder = len(paths_images) % duration
    
    # Create new video directory paths
    vpath = paths_images[0]
    vpath = vpath[: vpath.rfind("/")] # Original video directory path.
        # e.g., 'DATADIR/HMDB51png/YoYo/v_YoYo_g25_c05
    new_vpaths = [vpath.replace("HMDB51png", "HMDB51clip{}".format(duration)) + "_cc{:02d}".format(i + 1)
        for i in range(num_repeat + 1 if remainder != 0 else num_repeat)]
        # e.g., ['DATADIR/HMDB51clip79/YoYo/v_YoYo_g25_c05_cc01,
        #        'DATADIR/HMDB51clip79/YoYo/v_YoYo_g25_c05_cc02,
        #        'DATADIR/HMDB51clip79/YoYo/v_YoYo_g25_c05_cc03,
        #        'DATADIR/HMDB51clip79/YoYo/v_YoYo_g25_c05_cc04]
    
    # Create directories
    for v in new_vpaths:
        if not os.path.exists(v):
            os.makedirs(v)
            
    # Copy images
    rep_paths_images = paths_images[:-remainder] if remainder != 0 else paths_images[:-duration]
    rep_paths_images.extend(paths_images[-duration:])
    
    new_paths_images = []
    for v in new_vpaths:
        new_paths_images.extend([v + "/{:04d}.png".format(i + 1) for i in range(duration)])

    assert len(rep_paths_images) == len(new_paths_images), "{} != {}".format(len(rep_paths_images), len(new_paths_images))
    for i, j in zip(rep_paths_images, new_paths_images):
        shutil.copy(i, j)

    print("Done: {}.".format(vpath))

## Duration = 79

In [21]:
# New data directory: "DATADIR/HMDB51clip79"
duration = 79

In [22]:
# Loop for each class
for itr_cls in classnames:
    vpaths = videodir[itr_cls] # len = num of videos in the class
    nframes = numf[itr_cls] # len = num of videos in the class
    
    # Loop for each video in the class
    for itr_v, video_length in zip(vpaths, nframes):
        itr_v = itr_v.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]")
        imgpaths = sorted(glob(itr_v + "/*.png")) # paths to .png images in ascending order of frame numbers
        
        if video_length < duration:
            repeat_func(paths_images=imgpaths, duration=duration)

        elif video_length == duration:
            as_is_func(paths_images=imgpaths, duration=duration)

        elif video_length > duration:
            clip_func(paths_images=imgpaths, duration=duration)
            
        else:
            raise ValueError("video_length: {}, duration: {}".format(video_length, duration))

Done: /data/t-miyagawa/HMDB51png/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ri_med_3.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1

Done: /data/t-miyagawa/HMDB51png/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_2_brush_hair_h_nm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_2_brush_hair_h_nm_np1_ba_med_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_2_brush_hair_h_nm_np1_ba_med_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_cm_np2_ri_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_nm_np2_ba_goo_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_nm_np2_ba_goo_4.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_nm_np2_ri_goo_1.
Done: /data/t-miyagawa/HMDB51p

Done: /data/t-miyagawa/HMDB51png/cartwheel/Jennis_Bodenk_r_cartwheel_f_cm_np1_fr_med_3.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_le_med_2.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_le_med_5.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_ri_med_0.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_ri_med_4.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_ba_med_1.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_fr_med_3.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_fr_med_5.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_c

Done: /data/t-miyagawa/HMDB51png/catch/Goalkeeper_Training_Day_#_7_catch_f_nm_np1_ri_bad_4.
Done: /data/t-miyagawa/HMDB51png/catch/Handball_Passtraining_in_der_Zweiergruppe_(1)_catch_f_cm_np1_le_bad_1.
Done: /data/t-miyagawa/HMDB51png/catch/Handball_Passtraining_in_der_Zweiergruppe_(1)_catch_f_cm_np1_le_med_0.
Done: /data/t-miyagawa/HMDB51png/catch/How_to_Pass_in_Basketball_catch_f_cm_np1_le_bad_0.
Done: /data/t-miyagawa/HMDB51png/catch/How_to_Play_Wide_Receiver_in_Football_-_How_to_Catch_a_High_Ball_in_Football_catch_f_nm_np1_ri_goo_0.
Done: /data/t-miyagawa/HMDB51png/catch/How_to_Play_Wide_Receiver_in_Football_-_How_to_Catch_a_High_Ball_in_Football_catch_f_nm_np1_ri_med_1.
Done: /data/t-miyagawa/HMDB51png/catch/Phillies_dad_catches_foul_ball_before_daughter_tosses_it_away_catch_f_cm_np1_ba_bad_0.
Done: /data/t-miyagawa/HMDB51png/catch/Seldin_Lipovic_-_Willi_Weber__Torwarttraining__catch_f_cm_np1_ba_bad_7.
Done: /data/t-miyagawa/HMDB51png/catch/Seldin_Lipovic_-_Willi_Weber__Torwarttra

Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Bubbles_chew_h_cm_np1_fr_bad_1.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Bubbles_chew_h_cm_np1_fr_bad_2.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Wigger_chew_h_nm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Wigger_chew_h_nm_np1_fr_goo_1.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Wigger_chew_h_nm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/chew/Chewing_Gum_chew_h_nm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/chew/Chewing_Gum_chew_h_nm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/chew/Double_bubble_in_one_big_bubble_-D_chew_h_cm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/chew/Double_bubble_in_one_big_bubble_-D_chew_h_nm_np1_fr_goo_1.
Done: /data/t-miyagawa/HMDB51png/chew/Double_bubble_in_one_big_bubble_-D_chew_h_nm_np1_fr_goo_2.
Done: /data/t-miyagawa/HMDB51png/chew/Dubble_bubble_compilation_-_all_on_8_pieces_chew_h_nm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/chew/Dubble_bubble_compil

...

Done: /data/t-miyagawa/HMDB51png/walk/HP_PRISONER_OF_AZKABAN_walk_f_cm_np1_ri_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/HP_PRISONER_OF_AZKABAN_walk_f_nm_np1_ba_goo_6.
Done: /data/t-miyagawa/HMDB51png/walk/HP_PRISONER_OF_AZKABAN_walk_f_nm_np1_fr_med_7.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_1_walk_u_cm_np1_fr_goo_2.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_1_walk_u_cm_np1_fr_goo_3.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_1_walk_u_cm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_1_walk_u_cm_np1_le_med_4.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_1_walk_u_nm_np1_fr_med_7.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_2_walk_f_nm_np1_fr_med_6.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_2_walk_h_cm_np1_fr_med_3.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_2_walk_u_cm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/walk/Hitch_Part_2_walk_u_cm_np1_fr_med_10.
Done: /data/t-miyagawa/HMDB51png/walk/IamLegend_walk_f_cm_np1_le_med_

Done: /data/t-miyagawa/HMDB51png/walk/Return_of_the_King_11_walk_f_cm_np2_ri_med_3.
Done: /data/t-miyagawa/HMDB51png/walk/Return_of_the_King_5_walk_f_nm_np1_ba_med_1.
Done: /data/t-miyagawa/HMDB51png/walk/Return_of_the_King_8_walk_u_nm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/walk/RushHour2_walk_f_nm_np1_fr_med_10.
Done: /data/t-miyagawa/HMDB51png/walk/RushHour2_walk_u_cm_np1_fr_med_6.
Done: /data/t-miyagawa/HMDB51png/walk/RushHour2_walk_u_cm_np1_fr_med_9.
Done: /data/t-miyagawa/HMDB51png/walk/RushHour2_walk_u_cm_np2_ri_med_5.
Done: /data/t-miyagawa/HMDB51png/walk/SafeInPort_walk_f_nm_np1_ba_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/SafetyHarmHidesatHome_walk_f_cm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/SchoolRulesHowTheyHelpUs_walk_f_cm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/SellingasaCareer_walk_f_cm_np1_ri_bad_1.
Done: /data/t-miyagawa/HMDB51png/walk/SellingasaCareer_walk_f_nm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/Shadow_of_a_Doubt_wal

Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_21.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_29.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_37.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_38.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_41.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_f_nm_np1_fr_med_53.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_u_cm_np1_le_med_20.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_u_cm_np1_ri_med_10.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_u_cm_np1_ri_med_54.
Done: /data/t-miyagawa/HMDB51png/walk/TheLastManOnearth_walk_u_nm_np1_fr_med_55.
Done: /data/t-miyagawa/HMDB51png/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12.
Done: /data/t-miyagawa/H

Done: /data/t-miyagawa/HMDB51png/walk/Veoh_Alpha_Dog_2_walk_u_nm_np1_le_med_29.
Done: /data/t-miyagawa/HMDB51png/walk/WeddingCrashers_walk_f_nm_np1_fr_med_19.
Done: /data/t-miyagawa/HMDB51png/walk/WeddingCrashers_walk_u_cm_np1_ri_med_7.
Done: /data/t-miyagawa/HMDB51png/walk/WeddingCrashers_walk_u_nm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/walk/WeddingCrashers_walk_u_nm_np1_fr_med_16.
Done: /data/t-miyagawa/HMDB51png/walk/WeddingCrashers_walk_u_nm_np1_fr_med_20.
Done: /data/t-miyagawa/HMDB51png/walk/americanthrift1_walk_f_cm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/americanthrift1_walk_u_cm_np1_le_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/eddieizzardcircle_walk_f_nm_np1_le_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/gangboy256kb_walk_f_nm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/happy_go_lovely_walk_f_cm_np1_fr_med_11.
Done: /data/t-miyagawa/HMDB51png/walk/happy_go_lovely_walk_f_cm_np1_fr_med_13.
Done: /data/t-miyagawa/HMDB51png/walk/happy_go_lovely_walk

Done: /data/t-miyagawa/HMDB51png/wave/Gaye_waves_to_the_paparazzi_as_she_flees_New_York_after_stalking_Adam_Lambert_in_Central_Park_wave_u_cm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/Glory_wave_u_nm_np6_fr_med_33.
Done: /data/t-miyagawa/HMDB51png/wave/HP_PRISONER_OF_AZKABAN_wave_f_cm_np1_fr_med_12.
Done: /data/t-miyagawa/HMDB51png/wave/HP_PRISONER_OF_AZKABAN_wave_u_nm_np1_ri_med_39.
Done: /data/t-miyagawa/HMDB51png/wave/Hands_Up__Clap_Hands_And_Wave!_wave_f_cm_np1_ri_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/Hitch_Part_1_wave_u_nm_np1_fr_goo_1.
Done: /data/t-miyagawa/HMDB51png/wave/Hitch_Part_2_wave_h_nm_np1_fr_goo_12.
Done: /data/t-miyagawa/HMDB51png/wave/IamLegend_wave_u_nm_np1_ba_med_6.
Done: /data/t-miyagawa/HMDB51png/wave/IndianaJonesandTheTempleofDoom_wave_h_nm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/wave/KUNG_FU_HUSTLE_wave_f_nm_np1_fr_med_7.
Done: /data/t-miyagawa/HMDB51png/wave/L_cheln_Und_Winken_wave_h_cm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/wav

## Duration = 200

In [23]:
# New data directory: "DATADIR/HMDB51clip200"
duration = 200

In [24]:
# Loop for each class
for itr_cls in classnames:
    vpaths = videodir[itr_cls] # len = num of videos in the class
    nframes = numf[itr_cls] # len = num of videos in the class
    
    # Loop for each video in the class
    for itr_v, video_length in zip(vpaths, nframes):
        itr_v = itr_v.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]")
        imgpaths = sorted(glob(itr_v + "/*.png")) # paths to .png images in ascending order of frame numbers
        
        if video_length < duration:
            repeat_func(paths_images=imgpaths, duration=duration)

        elif video_length == duration:
            as_is_func(paths_images=imgpaths, duration=duration)

        elif video_length > duration:
            clip_func(paths_images=imgpaths, duration=duration)
            
        else:
            raise ValueError("video_length: {}, duration: {}".format(video_length, duration))

Done: /data/t-miyagawa/HMDB51png/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ri_med_3.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3.
Done: /data/t-miyagawa/HMDB51png/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1

Done: /data/t-miyagawa/HMDB51png/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_2_brush_hair_h_nm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_2_brush_hair_h_nm_np1_ba_med_1.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_2_brush_hair_h_nm_np1_ba_med_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_cm_np2_ri_goo_0.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_nm_np2_ba_goo_2.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_nm_np2_ba_goo_4.
Done: /data/t-miyagawa/HMDB51png/brush_hair/brushing_hair_brush_hair_f_nm_np2_ri_goo_1.
Done: /data/t-miyagawa/HMDB51p

Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_le_med_2.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_le_med_5.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_ri_med_0.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_ri_med_4.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_ba_med_1.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_fr_med_3.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_fr_med_5.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs_Cartwheel_Competition_cartwheel_f_cm_np1_fr_med_6.
Done: /data/t-miyagawa/HMDB51png/cartwheel/Jessica_and_Gregs

Done: /data/t-miyagawa/HMDB51png/catch/Goalkeeper_Training_Day_#_7_catch_f_cm_np1_ri_med_0.
Done: /data/t-miyagawa/HMDB51png/catch/Goalkeeper_Training_Day_#_7_catch_f_cm_np1_ri_med_1.
Done: /data/t-miyagawa/HMDB51png/catch/Goalkeeper_Training_Day_#_7_catch_f_nm_np1_ri_bad_4.
Done: /data/t-miyagawa/HMDB51png/catch/Handball_Passtraining_in_der_Zweiergruppe_(1)_catch_f_cm_np1_le_bad_1.
Done: /data/t-miyagawa/HMDB51png/catch/Handball_Passtraining_in_der_Zweiergruppe_(1)_catch_f_cm_np1_le_med_0.
Done: /data/t-miyagawa/HMDB51png/catch/How_to_Pass_in_Basketball_catch_f_cm_np1_le_bad_0.
Done: /data/t-miyagawa/HMDB51png/catch/How_to_Play_Wide_Receiver_in_Football_-_How_to_Catch_a_High_Ball_in_Football_catch_f_nm_np1_ri_goo_0.
Done: /data/t-miyagawa/HMDB51png/catch/How_to_Play_Wide_Receiver_in_Football_-_How_to_Catch_a_High_Ball_in_Football_catch_f_nm_np1_ri_med_1.
Done: /data/t-miyagawa/HMDB51png/catch/Phillies_dad_catches_foul_ball_before_daughter_tosses_it_away_catch_f_cm_np1_ba_bad_0.
Done: 

Done: /data/t-miyagawa/HMDB51png/chew/Bubble_gum!_XD_chew_h_cm_np1_fr_bad_2.
Done: /data/t-miyagawa/HMDB51png/chew/Bubble_gum!_XD_chew_h_cm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/chew/Bubble_gum!_XD_chew_h_cm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Bubbles_chew_h_cm_np1_fr_bad_0.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Bubbles_chew_h_cm_np1_fr_bad_1.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Bubbles_chew_h_cm_np1_fr_bad_2.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Wigger_chew_h_nm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Wigger_chew_h_nm_np1_fr_goo_1.
Done: /data/t-miyagawa/HMDB51png/chew/Bubblegum_Wigger_chew_h_nm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/chew/Chewing_Gum_chew_h_nm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/chew/Chewing_Gum_chew_h_nm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/chew/Double_bubble_in_one_big_bubble_-D_chew_h_cm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/chew/Double_bu

...

Done: /data/t-miyagawa/HMDB51png/walk/MeettheParents_walk_u_cm_np1_fr_med_5.
Done: /data/t-miyagawa/HMDB51png/walk/MeettheParents_walk_u_cm_np1_fr_med_7.
Done: /data/t-miyagawa/HMDB51png/walk/MeettheParents_walk_u_cm_np1_fr_med_8.
Done: /data/t-miyagawa/HMDB51png/walk/MeettheParents_walk_u_cm_np1_fr_med_9.
Done: /data/t-miyagawa/HMDB51png/walk/MeettheParents_walk_u_nm_np1_fr_med_6.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE1_walk_f_nm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE1_walk_f_nm_np1_le_med_3.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE1_walk_u_nm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE1_walk_u_nm_np1_fr_bad_1.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE1_walk_u_nm_np1_fr_med_5.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE2_walk_f_nm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/NOVA_ELEGANTUNIVERSE2_walk_u_nm_np1_ri_med_3.
Done: /data/t-miyagawa/HMDB

Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_ba_bad_18.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_fr_bad_89.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_fr_med_51.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_fr_med_52.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_fr_med_53.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_fr_med_7.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_fr_med_94.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np2_ri_med_14.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_cm_np3_ba_med_124.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_nm_np1_ba_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_nm_np1_fr_bad_57.
Done: /data/t-miyagawa/HMDB51png/walk/TheBoondockSaints_walk_f_nm_np1_fr_med_21.
Done: /data/t-miyagawa/HMDB51

Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_f_cm_np1_ri_med_16.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_f_nm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_f_nm_np1_fr_med_11.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_f_nm_np1_fr_med_20.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_f_nm_np1_fr_med_8.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_u_cm_np1_ba_med_15.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_u_cm_np1_fr_med_13.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_u_cm_np1_fr_med_14.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_u_cm_np1_fr_med_19.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_u_cm_np1_ri_med_4.
Done: /data/t-miyagawa/HMDB51png/walk/The_House_on_Haunted_Hill_walk_u_cm_np1_ri_med_6.
Done: /data/t-miyagawa/HM

Done: /data/t-miyagawa/HMDB51png/walk/prelinger_they_grow_up_so_fast_1_walk_f_nm_np1_fr_bad_11.
Done: /data/t-miyagawa/HMDB51png/walk/prelinger_they_grow_up_so_fast_1_walk_f_nm_np1_fr_med_5.
Done: /data/t-miyagawa/HMDB51png/walk/preparationoffoods256kb_walk_f_cm_np1_ri_med_1.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_f_nm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_f_nm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_f_nm_np1_fr_med_4.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_u_nm_np1_fr_med_13.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_u_nm_np1_fr_med_8.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_u_nm_np1_le_med_15.
Done: /data/t-miyagawa/HMDB51png/walk/prideandprejudice1_walk_u_nm_np1_le_med_9.
Done: /data/t-miyagawa/HMDB51png/wave/20060723sfjffbartsinger_wave_f_cm_np1_ba_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/21_wave_u_nm_np1_fr_goo_5.
Don

Done: /data/t-miyagawa/HMDB51png/wave/TrumanShow_wave_f_nm_np1_fr_med_26.
Done: /data/t-miyagawa/HMDB51png/wave/TrumanShow_wave_u_nm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/TrumanShow_wave_u_nm_np1_fr_med_25.
Done: /data/t-miyagawa/HMDB51png/wave/UNREPEATABLE_wave_f_nm_np1_fr_med_5.
Done: /data/t-miyagawa/HMDB51png/wave/We_Wave_-_Premiere_wave_f_cm_np2_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/WeddingCrashers_wave_u_nm_np1_fr_goo_0.
Done: /data/t-miyagawa/HMDB51png/wave/Wie_man_winkt!!_wave_u_cm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_Male_Choir_wave_u_cm_np1_fr_med_0.
Done: /data/t-miyagawa/HMDB51png/wave/_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_Male_Choir_wave_u_cm_np1_fr_med_1.
Done: /data/t-miyagawa/HMDB51png/wave/_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_Male_Choir_wave_u_cm_np1_fr_med_2.
Done: /data/t-miyagawa/HMDB51png/wave/eddieizzardcircle_wave_u_nm_np1_ri_med_8.
Done: /data/t-miyagawa/HMDB51png/

### Operation Confirmation

In [51]:
duration = 79

# Are all clipped videos has duration 79?
a = sorted(glob("DATADIR/HMDB51clip{}/*".format(duration)))
b = sorted(glob("DATADIR/HMDB51clip{}/*/*".format(duration)))
for v in a:
    for vv in b:
        if vv.find(v) != -1:
            print(len(glob(vv.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]") + "/*.png")))
            assert len(glob(vv.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]") + "/*.png")) == duration

print("It seems there are no problems in the video lengths. And...")

# Are all videos in HMDB51png included in HMDB51clip79?
for _, i in videodir.items():
    for ii in i:
        ii = ii.replace("png/", "clip{}/".format(duration))
        tmp = False
        for j in b:
            if j.find(ii) != -1:
                tmp = True
        assert tmp

print("It seems there is no video missed.")

79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
7

79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
7

79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
7

79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
7

79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
It seems there are no problems in the video lengths.
It seems there is no video missed.


In [52]:
duration = 200

# Are all clipped videos has duration 200?
a = sorted(glob("DATADIR/HMDB51clip{}/*".format(duration)))
b = sorted(glob("DATADIR/HMDB51clip{}/*/*".format(duration)))
for v in a:
    for vv in b:
        if vv.find(v) != -1:
            print(len(glob(vv.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]") + "/*.png")))
            assert len(glob(vv.replace("[", "[[").replace("]", "[]]").replace("[[", "[[]") + "/*.png")) == duration

print("It seems there are no problems in the video lengths. And...")
            
# Are all videos in HMDB51png included in HMDB51clip200?
for _, i in videodir.items():
    for ii in i:
        ii = ii.replace("png/", "clip{}/".format(duration))
        tmp = False
        for j in b:
            if j.find(ii) != -1:
                tmp = True
        assert tmp            

print("It seems there is no video missed.")

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
